Based on code by Miguel Sotaquirá (https://www.youtube.com/watch?v=0mOV4plF2Xo)

# 1. Read data set

### Function to read the data set

In [ ]:
import numpy as np
import os
import gzip

def load_set(path, data_type='train'):


    category_path = os.path.join(path, '%s-labels-idx1-ubyte.gz' % data_type)
    path_images = os.path.join(path, '%s-images-idx3-ubyte.gz' % data_type)
    
    with gzip.open(category_path, 'rb') as category_path_file:
        tags = np.frombuffer(category_path_file.read(), dtype=np.uint8, offset=8)

    with gzip.open(path_images, 'rb') as path_images_file:
        images = np.frombuffer(path_images_file.read(), dtype=np.uint8, offset=16).reshape(len(tags), 784)

    return images, tags

### Google drive access

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

path_drive = 'gdrive/MyDrive/master/ciencia_de_datos/tareas/intro_google_colab/fashion_mnist_data'

X_train, Y_train = load_set(path_drive, data_type='train')
X_test, Y_test = load_set(path_drive, data_type='test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Reshape data size to fit TPU requirements

In [ ]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

# 2. Model Creation with TensorFlow 2 + Keras

### Import Tensorflow 2 (Keras already included)

In [ ]:
%tensorflow_version 2.x   # for checking version 2
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x   # for checking version 2`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Versión de TensorFlow: 2.4.1


### Model creation

In [ ]:
tf.random.set_seed(200)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))

# Print model information
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_15 (Batc (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 7, 7, 128)        

# 3. Training with TPU

### Choose type training (CPU, GPU or TPU) by leave uncommented the one you want

In [ ]:
#device_name = '/cpu:0'
#device_name = tf.test.gpu_device_name()
device_name = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])



if type(device_name) is tf.distribute.cluster_resolver.TPUClusterResolver:
  tpu = device_name
  print('Training type TPU with "', tpu.cluster_spec().as_dict()['worker'], '"' )
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  with tpu_strategy.scope():
    tf.random.set_seed(200)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
    model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
    model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Activation('softmax'))
    
    x_train = X_train.astype(np.float32)
    x_test = X_test.astype(np.float32)
    y_train = Y_train.astype(np.float32)
    y_test = Y_test.astype(np.float32)
elif "GPU" in device_name: 
  print('Training type GPU with "', device_name, '"' )
else:
  print('Training type CPU with "', device_name, '"' )
    



Training type TPU with " ['10.65.91.146:8470'] "


INFO:tensorflow:Initializing the TPU system: grpc://10.65.91.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.65.91.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### Compile model

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Training with TPU

In [ ]:
import timeit

def training():

  if type(device_name) is tf.distribute.cluster_resolver.TPUClusterResolver:
    model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=128,epochs=2,verbose=1)    
  else:
    with tf.device(device_name):
      model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

training_time = timeit.timeit('training()', number=1, setup='from __main__ import training')

Epoch 1/2
468/468 [==============================] - 14s 20ms/step - loss: 1.0560 - accuracy: 0.7035 - val_loss: 0.3980 - val_accuracy: 0.8653
Epoch 2/2
468/468 [==============================] - 8s 17ms/step - loss: 0.4119 - accuracy: 0.8528 - val_loss: 0.3673 - val_accuracy: 0.8761


In [ ]:
print('Training time: ' + str(training_time) + ' seconds')

Training time: 24.512402483000187 seconds
